In [1]:
!pip install vadersentiment
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 11.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 1.8 MB/s 
     |████████████████████████████████| 88 kB 3.3 MB/s 
     |████████████████████████████████| 5.2 MB 16.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 57.1 MB/s 
     |████████████████████████████████| 85 kB 6.6 MB/s 
     |████████████████████████████████| 5.5 MB 46.9 MB/s 
     |████████████████████████████████| 1.3 MB 65.6 MB/s 
     |████████████████████████████████| 163 kB 67.2 MB/s 
     |████████████████████████████████| 7.6 MB 56.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
  Created wheel for hdbscan: filename=hdbsca

In [2]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
np.random.seed(10)
import nltk
from bertopic import BERTopic
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import seaborn as sns
import plotly.express as px
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
def clean_text(text):
    text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    text = re.sub(r'\'\w+', '', text) 
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = re.sub('[^a-zA-Z0-9 \.]', '', text)
    return text

In [4]:
def lemmatize(text):
  return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    if type(text)!=str:
      return ''
    result = []
    text = text.replace('[removed]', '').replace('[deleted]', '')
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    return ' '.join(result)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd
file_name = "/content/drive/MyDrive/cleaned_reddit_posts.csv"
df = pd.read_csv(file_name)
df.head()

,created_utc,cleaned_title,cleaned_comments,cleaned_url_content
0,2020-01-01 00:00:08,thousands of indians ushered in the new year b...,...because most indians are not muslims. the m...,new delhi thousands of indians ushered in the ...
1,2020-01-01 00:21:23,a scary unidentified virus is spreading in china,...ah a fellow time traveler....well good is t...,dozens of residents of wuhan the capital city ...
2,2020-01-01 00:31:16,kim jong un warns hostile us policy means ther...,this guy...trump will comfort kim....being tha...,washington cnn north korean leader kim jong un...
3,2020-01-01 00:53:25,north korean leader to end missile test ban cl...,...that a choice of course. we can agree that ...,kim jongun has signalled that north korea will...
4,2020-01-01 00:53:55,snapchat has stopped working for users around ...,oh the humanity ...omg....what will teenagers ...,for free real time breaking news alerts sent s...


In [8]:
i = 9
unclean = "https://www.theguardian.com/australia-news/bushfires ‘Disaster’s in the recovery’: bushfire survivors still waiting for homes"
clean = clean_text(unclean)
print(f"Uncleaned post: {unclean}")
print(f"Lightly cleaned post: {clean}")

Uncleaned post: https://www.theguardian.com/australia-news/bushfires ‘Disaster’s in the recovery’: bushfire survivors still waiting for homes
Lightly cleaned post:  disasters in the recovery bushfire survivors still waiting for homes


In [12]:
titles = df["cleaned_title"].astype(str).to_list()
# dates = df['created_utc']

# topic_model = BERTopic(min_topic_size=100, n_gram_range=(1,3), nr_topics='auto', verbose=True)
# topics, _ = topic_model.fit_transform(titles)
# topic_model = BERTopic.load("/content/drive/MyDrive/title_model_2")

In [13]:
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
docs = titles
topic_model = BERTopic(verbose=True, n_gram_range=(1, 3),nr_topics='auto',min_topic_size=100)
topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/3104 [00:00<?, ?it/s]

2022-11-06 21:55:07,052 - BERTopic - Transformed documents to Embeddings
2022-11-06 21:57:24,213 - BERTopic - Reduced dimensionality
2022-11-06 21:57:37,082 - BERTopic - Clustered reduced embeddings
2022-11-06 21:58:08,404 - BERTopic - Reduced number of topics from 133 to 45


In [14]:
coherence

0.4879954204932485

In [ ]:
topic_model.visualize_barchart(top_n_topics=4, n_words=10, height=400)

In [ ]:
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,59951,-1_to_in_of_the
1,0,4461,0_china_coronavirus_wuhan_virus
2,1,3356,1_israel_israeli_gaza_palestinian
3,2,2750,2_ukraine_russia_putin_russian
4,3,2449,3_vaccine_covid_covid vaccine_vaccines
5,4,2055,4_police_black_shooting_in
6,5,1866,5_johnson_boris_boris johnson_brexit
7,6,1854,6_covid_variant_omicron_uk
8,7,1844,7_space_nasa_earth_moon
9,8,1842,8_taliban_afghanistan_afghan_kabul


In [ ]:
topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)

[('ukraine', 0.03308394915474808),
 ('russia', 0.0196508324449488),
 ('putin', 0.017988323084516054),
 ('russian', 0.017627692385589117),
 ('invasion', 0.015556451754282117),
 ('of ukraine', 0.011333650707822057),
 ('invasion of ukraine', 0.011039977893549036),
 ('invasion of', 0.010906581920756895),
 ('russian invasion', 0.009985062206547156),
 ('kyiv', 0.009271725831019055)]

In [ ]:
fig = topic_model.visualize_topics()
fig

In [ ]:
df['topic'] = df['cleaned_title'].apply(lambda x: topic_model.find_topics(x)[0][0])
df.head()

,created_utc,cleaned_title,cleaned_comments,cleaned_url_content,topic
0,2020-01-01 00:00:08,thousands of indians ushered in the new year b...,...because most indians are not muslims. the m...,new delhi thousands of indians ushered in the ...,93
1,2020-01-01 00:21:23,a scary unidentified virus is spreading in china,...ah a fellow time traveler....well good is t...,dozens of residents of wuhan the capital city ...,0
2,2020-01-01 00:31:16,kim jong un warns hostile us policy means ther...,this guy...trump will comfort kim....being tha...,washington cnn north korean leader kim jong un...,11
3,2020-01-01 00:53:25,north korean leader to end missile test ban cl...,...that a choice of course. we can agree that ...,kim jongun has signalled that north korea will...,11
4,2020-01-01 00:53:55,snapchat has stopped working for users around ...,oh the humanity ...omg....what will teenagers ...,for free real time breaking news alerts sent s...,24


In [ ]:
df['cleaned_comments'] = df['cleaned_comments'].map(preprocess)
df.head()

,created_utc,cleaned_title,cleaned_comments,cleaned_url_content,topic,created_dt_month
0,2020-01-01 00:00:08,thousands of indians ushered in the new year b...,indians muslims million muslim feel real citiz...,new delhi thousands of indians ushered in the ...,93,01-2020
1,2020-01-01 00:21:23,a scary unidentified virus is spreading in china,fellow time traveler good turn gameplay weeks ...,dozens of residents of wuhan the capital city ...,0,01-2020
2,2020-01-01 00:31:16,kim jong un warns hostile us policy means ther...,guy trump comfort kim intentions denuking mean...,washington cnn north korean leader kim jong un...,11,01-2020
3,2020-01-01 00:53:25,north korean leader to end missile test ban cl...,choice course agree big problem north korea pu...,kim jongun has signalled that north korea will...,11,01-2020
4,2020-01-01 00:53:55,snapchat has stopped working for users around ...,humanity omg teenagers world snap corner carpe...,for free real time breaking news alerts sent s...,24,01-2020


In [ ]:
df.to_csv("reddit_data_with_topics.csv", index=False)

In [ ]:
sentiment_analyzer = SentimentIntensityAnalyzer()
df["comment_sentiment"] = df['cleaned_comments'].apply(lambda x: sentiment_analyzer.polarity_scores(x)["compound"])

In [ ]:
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

In [ ]:
# df["comment_sentiment"] = df['cleaned_comments'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
sentiment_df = df[["created_dt_month", "topic", "comment_sentiment"]].groupby(["created_dt_month", "topic"]).mean()
sentiment_df.head()

comment_sentiment
created_dt_month topic                   
01-2020          -1             -0.652650
                  0             -0.147076
                  1             -0.156931
                  2             -0.212218
                  3              0.021866

In [ ]:
sentiment_df = sentiment_df.reset_index()

In [ ]:
map_topic = freq[["Topic", "Name"]]
map_topic.columns = ['topic', 'name']
merged_df = sentiment_df.merge(map_topic, on=['topic'])

In [ ]:
merged_df["created_dt_month"] = pd.to_datetime(merged_df["created_dt_month"])
merged_df = merged_df.sort_values("created_dt_month")
merged_df = merged_df[merged_df['topic']!=-1]
merged_df.head()

,created_dt_month,topic,comment_sentiment,name
616,2020-01-01,30,-0.111050,30_zealand_new zealand_new_earthquake
1548,2020-01-01,77,0.185408,77_thunberg_greta_greta thunberg_nobel
596,2020-01-01,29,-0.520517,29_plane_crash_iran_boeing
136,2020-01-01,6,-0.965200,6_covid_variant_omicron_uk
1568,2020-01-01,78,-0.229304,78_elephants_elephant_zoo_endangered


In [ ]:
plot_df = merged_df[merged_df['topic'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
# plot_df["created_dt_month"] = pd.to_datetime(plot_df["created_dt_month"])
fig = px.line(plot_df, x="created_dt_month", y="comment_sentiment", color="name")
fig.show()
# sns.lineplot(data=plot_df, x="created_dt_month", y="comment_sentiment", hue="name")

In [ ]:
merged_df.to_csv("comment_sentiment.csv", index=False)

In [ ]:
df[(df["topic"]==2) & (df["created_dt_month"]=="04-2020")]["cleaned_comments"][24483]

'think minorities happy head yes didnt win world war shouldn mention come back tyranny back assad tyranny good glad clear think syrian people secretly happy assad dictatorship convince revolt look like share amp link load faster google amp want visit normal page instead elections freedom cool cause arab spring believe syrians incompetent want freedom think stupid try guess near russian base tartus syria understand play war game reason near syria weve country try overthrow government'

In [ ]:
sentiment_analyzer.polarity_scores(a)

{'neg': 0.122, 'neu': 0.59, 'pos': 0.288, 'compound': 0.9084}

Sentiment(polarity=0.1517857142857143, subjectivity=0.34649470899470897)